## 1. 触摸事件相关方法
### 1.1 dispatchTouchEvent
- 该方法主要是用来进行事件分发和传递的，当返回 true 的时候代表自己去处理，把事件传递给自己，否则就传递给其他的 view
- 该方法也是触摸事件第一个执行的方法，后续的几个是否执行都取决于它

### 1.2 onInterceptTouchEvent
- 这个方法主要是 ViewGroup 特有的，用来做触摸事件拦截的，默认返回 false
- 方法总结：
  - 主要是用来做事件分发过程中的拦截的，相当于一个拦截器
  - 如果返回 false 或者 super，则事件继续传递，事件所经过的每一层的 viewGroup 都会去调用该方法来询问是否拦截
  - 如果返回 true，则代表拦截该事件，停止传递给子 view，会走自己的 onTouchEvent 事件
  - 不像 onTouchEvent 是否拦截取决于 down 事件，该方法每个事件都可以去做拦截
  - 事件一经拦截，后续 move、up 事件都直接交给 onTouchEvent，不会重新去询问是否拦截(即不再调用 onInterceptTouchEvent)
  - 事件被拦截后，子 view 会接收到一个 cancel 事件，来恢复之前的状态，结束当前事件流

### 1.3 requestDisallowInterceptTouchEvent
- 这个方法也是用来做事件拦截的，也是 ViewGroup 专有的方法，不过一般是在子 view 中来调用的
- 当一个子 view 不希望它的父 view 来通过 onInterceptTouchEvent 方法拦截事件的时候，调用该方法即可实现事件的传递和接管，并且在整个事件流中，父 view 以及再往上的父 view 都要遵守该规则

### 1.4 onTouch
- 触摸事件，方便给开发者调用的
- 当一个触摸事件被分发到一个 view 的时候，就会调用该方法，它是在事件传递到 onTouchEvent 之前被调用的

### 1.5 onTouchEvent
- 该方法就是真正用来处理触摸事件的最后调用的方法

### 1.6 onClick
- 我们最熟悉的点击事件了，它也属于触摸事件的一个内容，有一点需要注意就是它是在 onTouchEvent 的 UP 事件里面执行的

---

## onTouch、onTouchEvent、onClick执行顺序
- 执行顺序：onTouch —> onTouchEvent —> onClick

```
# dispatchTouchEvent 源码
# onTouch 优先于 onTouchEvent 执行
public boolean dispatchTouchEvent(MotionEvent event) {
    // ......
    boolean result = false;
    // ......
    if (onFilterTouchEventForSecurity(event)) {
        // ......
        //noinspection SimplifiableIfStatement
        ListenerInfo li = mListenerInfo;
        if (li != null && li.mOnTouchListener != null
                && (mViewFlags & ENABLED_MASK) == ENABLED
                && li.mOnTouchListener.onTouch(this, event)) {
            result = true;
        }

        if (!result && onTouchEvent(event)) {
            result = true;
        }
    }
    // ......
    return result;
}
```

```
# onTouchEvent 源码
# performClick 里面就是执行的 onClick 事件
# onClick 是在 onTouchEvent 的 UP 事件里面才执行的
public boolean onTouchEvent(MotionEvent event) {
    //......
    if (clickable || (viewFlags & TOOLTIP) == TOOLTIP) {
        switch (action) {
            case MotionEvent.ACTION_UP:
                //......
                if ((mPrivateFlags & PFLAG_PRESSED) != 0 || prepressed) {
                    //......
                    if (!mHasPerformedLongPress && !mIgnoreNextUpEvent) {
                        // This is a tap, so remove the longpress check
                        removeLongPressCallback();

                        // Only perform take click actions if we were in the pressed state
                        if (!focusTaken) {
                            // Use a Runnable and post this rather than calling
                            // performClick directly. This lets other visual state
                            // of the view update before click actions start.
                            if (mPerformClick == null) {
                                mPerformClick = new PerformClick();
                            }
                            if (!post(mPerformClick)) {
                                performClick();
                            }
                        }
                    }
                    //......
                }
                mIgnoreNextUpEvent = false;
                break;
            case MotionEvent.ACTION_DOWN:
                //......
                break;
            case MotionEvent.ACTION_CANCEL:
                //......
                break;
            case MotionEvent.ACTION_MOVE:
                //......
                break;
        }
        return true;
    }
    return false;
}
```

---

## 触摸事件传递顺序
![image](page1.png)

- 上图一共分为三层，Activity ——> ViewGroup ——> View
- 其中 ViewGroup 比其他两个多了一个 onInterceptTouchEvent 方法
- 箭头代表了事件流的走向，箭头上的值则代表了该方法的返回值，绿色的消费框则代表了事件被消费掉，就此完结，不会再往下传递或者回溯

- 如果事件不被中断，则整个事件流就是一个完整的 U 型图
> 事件依次从 Activity 的 dispatchTouchEvent ——> ViewGroup 的 dispatchTouchEvent ——> ViewGroup 的 onInterceptTouchEvent ——> View 的 dispatchTouchEvent ——> View 的 onTouchEvent ——> ViewGroup 的 onTouchEvent ——> Activity 的 onTouchEvent 流转完成

- dispatchTouchEvent 和 onTouchEvent 一旦返回 true，事件就被消费掉了，该事件就消失了，不会往下传递也不会向上回溯
- dispatchTouchEvent 和 onTouchEvent 一旦返回 false，事件就会回溯到父控件的 onTouchEvent，说明自己不处理
> dispatchTouchEvent 返回 false 和 true 对于 Activity 来说都是一样，因为他是最顶层的事件接收者，而 ViewGroup 和 View 返回 super 则是向下传递，返回 false 就是向父控件的 onTouchEvent 回溯事件
> onTouchEvent 返回 super 代表向上回溯事件，返回 false 则代表自己不处理，所以也是向上回溯事件，如果最终都没消费，则 Activity 消费，事件消失

- 所有方法的 super 就是默认返回值，就是保证让整个事件流按照 U 型图走完
- onInterceptTouchEvent 默认返回 super，通过源码我们知道其实就是返回 false，默认是不去拦截事件的，这也符合常理，可以让子 view 有机会去捕获事件，返回 true 则代表拦截了这个事件，交给自己的 onTouchEvent 去处理，ViewGroup 的 dispatchTouchEvent 的 super 默认实现就是调用自己的 onInterceptTouchEvent，这也就可以保证事件有机会分发到自己的 onTouchEvent
- dispatchTouchEvent 和 onTouchEvent 都是以 Down 事件为基准，来判断后续事件是否经过自己，也就是自己消费，如果 Down 事件返回了 false 或者 super，则后续事件都不再经过自己了，包括 move，up，只有返回 true 的时候，后续事件才会经过自己

---

## 关于 ACTION_DOWN 的理解
- view 的 onTouchEvent 中 DOWN 事件返回 true
  - 可以看到比上面直接在方法最后返回 true 多出了最后一行日志，其他完全一致，这是因为只有 down 返回了 true，仅仅是让后续事件经过自己，但是 move、up 事件返回的还是 super，而 ViewGroup 的 onTouchEvent 事件已经被跳过，所以 up 事件回溯到 Activity 了。而在 move 和 up 事件中返回 true 则没有 down 返回 true 的作用
```
E/MainActivity: =====dispatchTouchEvent=====ACTION_DOWN
E/VG: dispatchTouchEvent: ======ACTION_DOWN
    onInterceptTouchEvent: ======ACTION_DOWN
E/V: dispatchTouchEvent: ======ACTION_DOWN
    onTouchEvent: ======ACTION_DOWN
E/MainActivity: =====dispatchTouchEvent=====ACTION_UP
E/VG: dispatchTouchEvent: ======ACTION_UP
    onInterceptTouchEvent: ======ACTION_UP
E/V: dispatchTouchEvent: ======ACTION_UP
    onTouchEvent: ======ACTION_UP
E/MainActivity: =====onTouchEvent=====ACTION_UP
```

- 在 view 的 dispatchTouchEvent 的 DOWN 返回 true
  - 事件走到 View 的 dispatchTouchEvent 后就停止了，因为这里返回了 true，代表事件在这里消费了，而后续的 UP 事件同样也是走到 view 的 dispatchTouchEvent，由于 up 返回的是 super，所以走了自身的 onTouchEvent的 up，然后这里返回的也是 super，所以又回溯给 Activity 的 onTouchEvent 的 up 了
```
E/MainActivity: =====dispatchTouchEvent=====ACTION_DOWN
E/VG: dispatchTouchEvent: ======ACTION_DOWN
    onInterceptTouchEvent: ======ACTION_DOWN
E/V: dispatchTouchEvent: ======ACTION_DOWN
E/MainActivity: =====dispatchTouchEvent=====ACTION_UP
E/VG: dispatchTouchEvent: ======ACTION_UP
    onInterceptTouchEvent: ======ACTION_UP
E/V: dispatchTouchEvent: ======ACTION_UP
    onTouchEvent: ======ACTION_UP
E/MainActivity: =====onTouchEvent=====ACTION_UP
```

### 总结
> 收到了ACTION_DOWN，就会收到 ACTION_MOVE、ACTION_UP 等后续的事件的前提条件就是必须消费该事件，也就是返回 true
> 如果仅仅是在 ACTION_DOWN 返回 true，其他事件返回 super，则其他事件没被消费，会继续向上回溯，但是一定会经过消费控件本身

---




## 关于 onInterceptTouchEvent 的理解
- onInterceptTouchEvent 不会像 onTouchEvent 一样必须在 DOWN 里面决定是否消费，它是用来做拦截的，也就相当于一个分流器，所以在它的所有事件都可以去分流
- 比如我们手指按下列表中的一个 view，然后过一会去滑动，这个时候依然会走 onInterceptTouchEvent 方法，不过走的就是 move 了(前提是你在 DOWN 事件中没有去拦截，也就是让事件向子 view 传递)，这个时候你就可以在 move 事件中去做拦截，来进行列表的一个滑动，这种设计是符合正常逻辑的

- 在 down 中返回 true
```
# 直接被拦下了，走了自己的 onTouchEvent，由于返回了 super，所以事件回溯给 Activity
E/MainActivity: =====dispatchTouchEvent=====ACTION_DOWN
E/VG: dispatchTouchEvent: ======ACTION_DOWN
    onInterceptTouchEvent: ======ACTION_DOWN
E/VG: onTouchEvent: ======ACTION_DOWN
E/MainActivity: =====onTouchEvent=====ACTION_DOWN
E/MainActivity: =====dispatchTouchEvent=====ACTION_UP
E/MainActivity: =====onTouchEvent=====ACTION_UP
```

- 在 move 中返回 true（onTouchEvent 中返回 true，因为 DOWN 事件代表拦截，如果不返回 true，后续 move 就不会再经过 ViewGroup）
```
E/MainActivity: =====dispatchTouchEvent=====ACTION_DOWN
E/VG: dispatchTouchEvent: ======ACTION_DOWN
    onInterceptTouchEvent: ======ACTION_DOWN
E/V: dispatchTouchEvent: ======ACTION_DOWN
    onTouchEvent: ======ACTION_DOWN
E/VG: onTouchEvent: ======ACTION_DOWN
E/MainActivity: =====dispatchTouchEvent=====ACTION_MOVE
E/VG: dispatchTouchEvent: ======ACTION_MOVE
    onTouchEvent: ======ACTION_MOVE
E/MainActivity: =====dispatchTouchEvent=====ACTION_MOVE
E/VG: dispatchTouchEvent: ======ACTION_MOVE
    onTouchEvent: ======ACTION_MOVE
E/MainActivity: =====dispatchTouchEvent=====ACTION_UP
E/VG: dispatchTouchEvent: ======ACTION_UP
    onTouchEvent: ======ACTION_UP
```

---

## 关于 ACTION_CANCEL 的出现场景
- 在上述 onInterceptTouchEvent 场景中，其实还少了一步，就是 View 的 onTouchEvent 的也要返回 true，这样就代表一开始是点击，然后停顿一下，手指去滑动，变成滑动了，通过 onInterceptTouchEvent 去拦截，这个时候 View 就会收到一个 ACTION_CANCEL 事件来恢复自己初始按下的状态
  - 事件传递到 view 的 onTouchEvent，就在被系统认为消费的时候，然后手指滑动，进而 ViewGroup 去拦截该滑动，这个时候就会额外触发一个 ACTION_CANCEL 来传递给子 view 进而恢复子 view 的状态
```
E/MainActivity: =====dispatchTouchEvent=====ACTION_DOWN
E/VG: dispatchTouchEvent: ======ACTION_DOWN
    onInterceptTouchEvent: ======ACTION_DOWN
E/V: dispatchTouchEvent: ======ACTION_DOWN
    onTouchEvent: ======ACTION_DOWN
E/MainActivity: =====dispatchTouchEvent=====ACTION_MOVE
E/VG: dispatchTouchEvent: ======ACTION_MOVE
    onInterceptTouchEvent: ======ACTION_MOVE
E/V: dispatchTouchEvent: ======ACTION_CANCEL
    onTouchEvent: ======ACTION_CANCEL
E/MainActivity: =====dispatchTouchEvent=====ACTION_MOVE
E/VG: dispatchTouchEvent: ======ACTION_MOVE
    onTouchEvent: ======ACTION_MOVE
E/MainActivity: =====dispatchTouchEvent=====ACTION_UP
E/VG: dispatchTouchEvent: ======ACTION_UP
E/VG: onTouchEvent: ======ACTION_UP
```

---

### [参考代码](https://github.com/binbinqq86/touchEvent)

---